In [1]:
from parcels import FieldSet

In [2]:
# Names of files containing the data
velocity_file = "cmems_mod_glo_phy_anfc_merged-uv_PT1H-i_multi-vars_74.08W-72.42W_37.67S-35.42S_0.49m_2023-01-01-2023-12-31.nc"
filenames = {
    'U': velocity_file,
    'V': velocity_file
}

In [3]:
# Names of the velocity component data in the file
variables = {
    'U': 'uo',
    'V': 'uo'
}

In [4]:
# Names of the dimensions in the file
dimensions = {
    'lat': 'latitude',
    'lon': 'longitude',
    'time': 'time'
}
# Create the FieldSet object
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

In [5]:
whos

Variable        Type        Data/Info
-------------------------------------
FieldSet        type        <class 'parcels.fieldset.FieldSet'>
dimensions      dict        n=3
fieldset        FieldSet    <parcels.fieldset.FieldSe<...>object at 0x7f1af3dacc50>
filenames       dict        n=2
variables       dict        n=2
velocity_file   str         cmems_mod_glo_phy_anfc_me<...>_2023-01-01-2023-12-31.nc


In [6]:
variables

{'U': 'uo', 'V': 'uo'}

In [7]:
FieldSet

parcels.fieldset.FieldSet

In [9]:
time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


In [5]:
from parcels import JITParticle, ParticleSet, Variable

# Define a new particleclass with Variable 'age' with initial value 0.
AgeParticle = JITParticle.add_variable(Variable("age", initial=0))

pset = ParticleSet(
    fieldset=fieldset,  # the fields that the particleset uses
    pclass=AgeParticle,  # define the type of particle
    lon=-73.1217,  # release longitude
    lat=-36.4137,  # release latitude
)

In [6]:
# load basic advection kernels
from parcels import AdvectionRK4

# Create a custom kernel which displaces each particle southward


def NorthVel(particle, fieldset, time):
    if time > 10 * 86400 and time < 10.2 * 86400:
        vvel = -1e-4
        particle_dlat += vvel * particle.dt


# Create a custom kernel which keeps track of the particle age (minutes)


def Age(particle, fieldset, time):
    particle.age += particle.dt / 3600


# define all kernels to be executed on particles using an (ordered) list
kernels = [Age, NorthVel, AdvectionRK4]

In [7]:
output_file = pset.ParticleFile(
    name="Test001.zarr",  # the name of the output file
    outputdt=3600,  # the time period between consecutive out output steps
    chunks=(1, 10),  # the chunking of the output file (number of particles, timesteps)
)

pset.execute(
    kernels,  # the kernel (which defines how particles move)
    runtime=86400 * 24,  # the total length of the run in seconds
    dt=300,  # the timestep of the kernel in seconds
    output_file=output_file,
)

INFO: Output files are stored in Test001.zarr.
 42%|██████████▉               | 871200.0/2073600.0 [00:07<00:10, 115288.29it/s]

FieldOutOfBoundError: Field sampled out-of-bound, at (-73.25740051269531, -37.692867279052734, 0.0)

 42%|██████████▉               | 875100.0/2073600.0 [00:19<00:10, 115288.29it/s]